In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Retinal_Vessel_Segmentation')
sys.path.append('/content/drive/MyDrive/Retinal_Vessel_Segmentation/libraries')

In [ ]:
pip install tensorboardx

     |████████████████████████████████| 122kB 19.7MB/s 


In [ ]:
import torch.backends.cudnn as cudnn
import torch.optim as optim
from tqdm import tqdm
import random,sys,time
import os
from os.path import join
import torch
from libraries.extract_patches import get_data_train
from libraries.losses.loss import *
from libraries.visualize import group_images, save_img
from libraries.common import *
from libraries.dataset import TrainDataset,TestDataset
from torch.utils.data import DataLoader
from config import parse_args
from libraries.logger import Logger, Print_Logger
from collections import OrderedDict
from libraries.metrics import Evaluate
import models
#from models import UNetFamily
#from test import Test

In [ ]:
#  Load the data and extract patches
def get_dataloader(val_ratio, batch_size):
    # patches_imgs_train, patches_masks_train = get_data_train(
    #     data_path_list = args.train_data_path_list,
    #     patch_height = args.train_patch_height,
    #     patch_width = args.train_patch_width,
    #     N_patches = args.N_patches,
    #     inside_FOV = args.inside_FOV #select the patches only inside the FOV  (default == False)
    # )

    patches_imgs_train = np.load('/content/drive/MyDrive/Retinal_Vessel_Segmentation/patches_imgs_train.npy')
    patches_masks_train = np.load('/content/drive/MyDrive/Retinal_Vessel_Segmentation/patches_masks_train.npy')

    val_ind = random.sample(range(patches_masks_train.shape[0]),int(np.floor(val_ratio*patches_masks_train.shape[0])))
    train_ind =  set(range(patches_masks_train.shape[0])) - set(val_ind)
    train_ind = list(train_ind)

    train_set = TrainDataset(patches_imgs_train[train_ind,...],patches_masks_train[train_ind,...],mode="train")
    train_loader = DataLoader(train_set, batch_size=batch_size,
                              shuffle=True, num_workers=4)

    val_set = TrainDataset(patches_imgs_train[val_ind,...],patches_masks_train[val_ind,...],mode="val")
    val_loader = DataLoader(val_set, batch_size=batch_size,
                            shuffle=False, num_workers=4)
    
    # Save some samples of feeding to the neural network
    # N_sample = min(patches_imgs_train.shape[0], 50)
    # save_img(group_images((patches_imgs_train[0:N_sample, :, :, :]*255).astype(np.uint8), 10),
    #           join(args.outf, args.save, "sample_input_imgs.png"))
    # save_img(group_images((patches_masks_train[0:N_sample, :, :, :]*255).astype(np.uint8), 10),
    #           join(args.outf, args.save,"sample_input_masks.png"))

    return train_loader,val_loader

# train 
def train(train_loader,net,criterion,optimizer,device):
    net.train()
    train_loss = AverageMeter()

    for batch_idx, (inputs, targets) in tqdm(enumerate(train_loader), total=len(train_loader)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss.update(loss.item(), inputs.size(0))
    log = OrderedDict([('train_loss',train_loss.avg)])
    return log

# val 
def val(val_loader,net,criterion,device):
    net.eval()
    val_loss = AverageMeter()
    evaluater = Evaluate()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in tqdm(enumerate(val_loader), total=len(val_loader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            val_loss.update(loss.item(), inputs.size(0))

            outputs = outputs.data.cpu().numpy()
            targets = targets.data.cpu().numpy()
            evaluater.add_batch(targets,outputs[:,1])
    log = OrderedDict([('val_loss', val_loss.avg), 
                       ('val_acc', evaluater.confusion_matrix()[1]), 
                       ('val_f1', evaluater.f1_score()),
                       ('val_auc_roc', evaluater.auc_roc())])
    return log

In [ ]:
setpu_seed(2021)
outf = "/content/drive/MyDrive/Retinal_Vessel_Segmentation/Experiments"
save = "UNet_vessel_seg"
save_path = os.path.join(outf, save)

device = torch.device("cuda" if torch.cuda.is_available() and True else "cpu")
cudnn.benchmark = True

log = Logger(save_path)
sys.stdout = Print_Logger(os.path.join(save_path,'train_log.txt'))
print('The computing device used is: ','GPU' if device.type=='cuda' else 'CPU')

net = models.UNetFamily.U_Net(1,2).to(device)
print("Total number of parameters: " + str(count_parameters(net)))
log.save_graph(net,torch.randn((1,1,64,64)).to(device).to(device=device))

N_epochs = 3
start_epoch = 1
criterion = CrossEntropyLoss2d() # Initialize loss function
optimizer = optim.Adam(net.parameters(), lr=0.0005)
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=N_epochs, eta_min=0)
val_ratio = 0.1
batch_size = 64

train_loader, val_loader = get_dataloader(val_ratio, batch_size) # create dataloader

The computing device used is:  GPU
Total number of parameters: 34525954
Architecture of Model have saved in Tensorboard!


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
best = {'epoch':0,'AUC_roc':0.5} # Initialize the best epoch and performance(AUC of ROC)
trigger = 0  # Early stop Counter

for epoch in range(start_epoch,N_epochs+1):
  print('\nEPOCH: %d/%d --(learn_rate:%.6f) | Time: %s' % \
            (epoch, N_epochs,optimizer.state_dict()['param_groups'][0]['lr'], time.asctime()))
  
  # train stage
  train_log = train(train_loader,net,criterion, optimizer,device)
  # val stage
  val_log = val(val_loader,net,criterion,device)

  log.update(epoch,train_log,val_log) # Add log information
  lr_scheduler.step()

  # Save checkpoint of latest and best model
  state = {'net': net.state_dict(),'optimizer':optimizer.state_dict(),'epoch': epoch}
  torch.save(state, join(save_path, 'latest_model.pth'))
  trigger += 1
  if val_log['val_auc_roc'] > best['AUC_roc']:
    print('\033[0;33mSaving best model!\033[0m')
    torch.save(state, join(save_path, 'best_model.pth'))
    best['epoch'] = epoch
    best['AUC_roc'] = val_log['val_auc_roc']
    trigger = 0

  print('Best performance at Epoch: {} | AUC_roc: {}'.format(best['epoch'],best['AUC_roc']))

  # early stopping
  early_stop = 6
  if not early_stop is None:
    if trigger >= early_stop:
      print("=> early stopping")
      break

  torch.cuda.empty_cache()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 32/32 [00:03<00:00, 10.00it/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 32/32 [00:03<00:00, 10.